<a href="https://colab.research.google.com/github/samyarsworld/Human-segmentation-deep-learning/blob/main/Human_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Segmentation

### 0. Clone Repository:
Clone the repository and move to the corresponding directory. The directory includes the two sub-directories of segmented images and their corresponding images and a helper function.

Repository address: https://github.com/samyarsworld/Human-segmentation-deep-learning

In [ ]:
!git clone https://github.com/samyarsworld/Human-segmentation-deep-learning
%cd Human-segmentation-deep-learning

### 1. Install Segmentation Models and Change Runtime:
Change the Colab runtime to support the latest available GPU (T4 at this time)

In [ ]:
!pip install -U segmentation-models-pytorch
!pip install -U git+https://github.com/albumentations-team/albumentations
!pip install -U opencv-contrib-python

**"opencv-contrib-python":** Refers to the Python package that contains the extra modules or extensions for the OpenCV (Open Source Computer Vision Library) that are not included in the standard OpenCV package. These extra modules provide additional functionality and tools for various computer vision tasks.  
**Albumentations:** Provides a wide range of image augmentation techniques that can be easily applied to image data, such as rotations, flips, color adjustments, blur, and more. The library is known for its speed and efficiency, making it suitable for real-time data augmentation during model training.  
**segmentation-models-pytorch:** Offers a collection of pre-trained models and various segmentation architectures that you can use for your own image segmentation projects. These models are often based on state-of-the-art deep learning architectures and can be fine-tuned for specific tasks or used as feature extractors.

### 3. Import Required Libraries and Functions:

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import cv2
# from helper_functions import accuracy_fn

### 4. Setup Initial Configurations:

### 5. Setup Augmentation Functions
These function are incorporated by referring to albumentation documentation : https://albumentations.ai/docs/

### 6. Create Custom Dataset

### 7. Batch the Dataset

### 8. Create The Segmentation Model

### 9. Create Train and Validation Function

### 10. Train Model

### 11. Inference and Predictions